In [ ]:
%env OPENAI_API_KEY="keyhere"

In [ ]:
#!/bin/bash

helm upgrade --install argo-rollouts argo-rollouts \
  --repo https://argoproj.github.io/argo-helm \
  --version 2.37.6 \
  --namespace argo-rollouts \
  --create-namespace \
  --wait

kubectl apply -f- <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  name: rollouts-demo
  labels:
    app: rollouts-demo
spec:
  replicas: 4
  selector:
    matchLabels:
      app: rollouts-demo
  template:
    metadata:
      labels:
        app: rollouts-demo
    spec:
      containers:
      - name: rollouts-demo
        image: argoproj/rollouts-demo:blue
        ports:
        - name: http
          containerPort: 8080
          protocol: TCP
  strategy:
    type: RollingUpdate
    rollingUpdate:
      maxSurge: 1
      maxUnavailable: 1
---
apiVersion: v1
kind: Service
metadata:
  name: rollouts-demo
  labels:
    app: rollouts-demo
spec:
  ports:
  - port: 80
    targetPort: http
    protocol: TCP
    name: http
  selector:
    app: rollouts-demo
apiVersion: gateway.networking.k8s.io/v1
kind: Gateway
metadata:
  name: gateway
spec:
  gatewayClassName: istio
  listeners:
  - name: default
    port: 80
    protocol: HTTP
    allowedRoutes:
      namespaces:
        from: All
---
kind: HTTPRoute
apiVersion: gateway.networking.k8s.io/v1beta1
metadata:
  name: rollouts-demo
spec:
  parentRefs:
    - name: gateway
  rules:
  - matches:
    - path:
        type: PathPrefix
        value: /  
    backendRefs:
    - name: rollouts-demo
      kind: Service
      port: 80
EOF

In [ ]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

from kagent.tools import BuiltInTool

model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
)

argo_rollout_agent = AssistantAgent(
    "ArgoAgent",
    description="An agent for debugging and executing argo tasks.",
    model_client=model_client,
    system_message="""
    You are an argo rollouts agent.
    Your job is to . DO NOT MAKE UP ADDITIONAL AND UNNECESSARY SUBTASKS.
    Your tools are kubectl argo rollouts:
    - kubectl argo rollouts create
    - kubectl argo rollouts get
    - kubectl argo rollouts pause
    - kubectl argo rollouts resume
    - kubectl argo rollouts promote
    - kubectl argo rollouts status
    - kubectl argo rollouts abort
    - verify installation
    """,
)

k8s_agent = AssistantAgent(
    "k8s_agent",
    description="An agent for k8s operations",
    tools=[BuiltInTool("k8s.apply_manifest")],
    model_client=model_client,
    system_message="""
    You are a k8s agent. You know how to interact with a Kubernetes cluster.
    """,
)

text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination


team = SelectorGroupChat(
    [argo_rollout_agent, k8s_agent],
    model_client=model_client,
    termination_condition=termination,
    allow_repeated_speaker=True,
)

task = "Create an Argo Rollout to deploy a new version of the demo application with the color purple using the Kubernetes Gateway API"

# Use asyncio.run(...) if you are running this in a script.
await Console(team.run_stream(task=task))


In [ ]:
print(team.dump_component().model_dump_json(indent=2))